In [1]:
import numpy as np
import pandas as pd
import json 
import ast

In [2]:
stream=pd.read_csv("/Users/manindersingh/Documents/Transactional Dataset/click_stream.csv")

In [3]:
transaction=pd.read_csv("/Users/manindersingh/Documents/Transactional Dataset/transactions.csv")

In [4]:
customer=pd.read_csv("/Users/manindersingh/Documents/Transactional Dataset/customer.csv")

In [5]:
product=pd.read_excel("/Users/manindersingh/Documents/Transactional Dataset/product.xlsx")

# Customer Behaviour

In [6]:
stream.head()

,session_id,event_name,event_time,event_id,traffic_source,event_metadata
0,fb0abf9e-fd1a-44dd-b5c0-2834d5a4b81c,HOMEPAGE,2019-09-06T15:54:32.821085Z,9c4388c4-c95b-4678-b5ca-e9cbc0734109,MOBILE,NaN
1,fb0abf9e-fd1a-44dd-b5c0-2834d5a4b81c,SCROLL,2019-09-06T16:03:57.821085Z,4690e1f5-3f99-42d3-84a5-22c4c4d8500a,MOBILE,NaN
2,7d440441-e67a-4d36-b324-80ffd636d166,HOMEPAGE,2019-09-01T12:05:10.322763Z,88aeaeb5-ec98-4859-852c-8abb483faf31,MOBILE,NaN
3,7d440441-e67a-4d36-b324-80ffd636d166,ADD_TO_CART,2019-09-01T12:06:33.322763Z,934e306e-ecc6-472f-9ccb-12c8536910a2,MOBILE,"{'product_id': 15315, 'quantity': 4, 'item_pri..."
4,7d440441-e67a-4d36-b324-80ffd636d166,BOOKING,2019-09-01T12:15:29.425431Z,9f4767a1-40fa-4c9c-9524-dfad18634d56,MOBILE,{'payment_status': 'Success'}


In [7]:
stream.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12833602 entries, 0 to 12833601
Data columns (total 6 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   session_id      object
 1   event_name      object
 2   event_time      object
 3   event_id        object
 4   traffic_source  object
 5   event_metadata  object
dtypes: object(6)
memory usage: 587.5+ MB


In [8]:
stream.isnull().sum()

session_id              0
event_name              0
event_time              0
event_id                0
traffic_source          0
event_metadata    8544062
dtype: int64

In [9]:
def safe_parse(x):
    try:
        if isinstance(x, str) and x != 'nan':
            return ast.literal_eval(x)  # Convert string to dictionary
        else:
            return x  # Return NaN or non-string as is
    except:
        return x  # If conversion fails, return the original value

# Apply the parsing function to the 'event_metadata' column
stream['event_metadata'] = stream['event_metadata'].apply(safe_parse)

In [10]:
stream['event_time']=pd.to_datetime(stream['event_time'])

In [11]:
stream=stream.sort_values(by=['session_id','event_time'])
event_sequence = stream.groupby('session_id')['event_name'].apply(lambda x: ' → '.join(x))

In [12]:
stream['payment_success'] = stream['event_metadata'].apply(lambda x: 1 if isinstance(x, dict) and x.get('payment_status') == 'Success' else 0)

In [13]:
payment_success = stream.groupby('session_id')['payment_success'].max()

In [14]:
first_event_date = stream.groupby('session_id')['event_time'].min()

In [15]:
payment_success_date = stream.groupby('session_id').apply(
    lambda x: x[x['payment_success'] == 1]['event_time'].min() if (x['payment_success'] == 1).any() else '')

/var/folders/4s/v2k7bm2d7vl1z46yc265hz3c0000gn/T/ipykernel_84198/1093817908.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  payment_success_date = stream.groupby('session_id').apply(


In [16]:
payment_success_date = pd.to_datetime(payment_success_date)
first_event_date = pd.to_datetime(first_event_date)

time_to_conversion = (payment_success_date - first_event_date).dt.days

In [17]:
customer_behaviour=pd.DataFrame({
    'session_id':event_sequence.index,
    'event_sequence':event_sequence.values,
    'payment_success': payment_success.values,
    'first_event_date': first_event_date.values,
    'first_payment_success_date': payment_success_date.values,
    'time_to_conversion': time_to_conversion.values})
# payment_success_date.values

In [18]:
customer_behaviour.head()

,session_id,event_sequence,payment_success,first_event_date,first_payment_success_date,time_to_conversion
0,00001f29-3ede-4445-95f0-0178934e32ba,HOMEPAGE → ADD_TO_CART → HOMEPAGE → CLICK → CL...,1,2019-05-17 11:51:52.276396,2019-05-19 20:17:04.276396,2.0
1,00003eca-954b-4150-aee1-63fc62f395cf,HOMEPAGE → PROMO_PAGE → ADD_TO_CART → BOOKING,1,2019-08-24 05:47:16.849738,2019-08-24 06:26:12.662344,0.0
2,0000425b-ddc8-4459-a240-6ba814dac132,HOMEPAGE → ADD_TO_CART → PROMO_PAGE → BOOKING,1,2019-02-05 13:38:59.490443,2019-04-28 10:08:01.490443,81.0
3,00005746-17ec-449c-9c5c-699a3ea9304a,HOMEPAGE → CLICK → ADD_TO_CART → SCROLL → SEAR...,1,2022-03-08 23:17:19.510057,2022-03-08 23:21:35.510057,0.0
4,0000649b-9514-4600-979b-42166a798996,HOMEPAGE → ADD_TO_CART → CLICK → ADD_PROMO → C...,1,2020-07-17 13:32:48.525079,2020-07-23 14:31:15.525079,6.0


In [19]:
customer_behaviour.isnull().sum()

session_id                        0
event_sequence                    0
payment_success                   0
first_event_date                  0
first_payment_success_date    79241
time_to_conversion            79241
dtype: int64

In [20]:
customer_behaviour['time_to_conversion'] = customer_behaviour['time_to_conversion'].fillna("")

In [21]:
customer_behaviour=customer_behaviour.merge(transaction[['session_id','customer_id']],on="session_id",how="inner")

# Product Summary

In [22]:
transaction.head()

,created_at,customer_id,booking_id,session_id,product_metadata,payment_method,payment_status,promo_amount,promo_code,shipment_fee,shipment_date_limit,shipment_location_lat,shipment_location_long,total_amount
0,2018-07-29T15:22:01.458193Z,5868,186e2bee-0637-4710-8981-50c2d737bc42,3abaa6ce-e320-4e51-9469-d9f3fa328e86,"[{'product_id': 54728, 'quantity': 1, 'item_pr...",Debit Card,Success,1415,WEEKENDSERU,10000,2018-08-03T05:07:24.812676Z,-8.227893,111.969107,199832
1,2018-07-30T12:40:22.365620Z,4774,caadb57b-e808-4f94-9e96-8a7d4c9898db,2ee5ead1-f13e-4759-92df-7ff48475e970,"[{'product_id': 16193, 'quantity': 1, 'item_pr...",Credit Card,Success,0,NaN,10000,2018-08-03T01:29:03.415705Z,3.013470,107.802514,155526
2,2018-09-15T11:51:17.365620Z,4774,6000fffb-9c1a-4f4a-9296-bc8f6b622b50,93325fb6-eb00-4268-bb0e-6471795a0ad0,"[{'product_id': 53686, 'quantity': 4, 'item_pr...",OVO,Success,0,NaN,10000,2018-09-18T08:41:49.422380Z,-2.579428,115.743885,550696
3,2018-11-01T11:23:48.365620Z,4774,f5e530a7-4350-4cd1-a3bc-525b5037bcab,bcad5a61-1b67-448d-8ff4-781d67bc56e4,"[{'product_id': 20228, 'quantity': 1, 'item_pr...",Credit Card,Success,0,NaN,0,2018-11-05T17:42:27.954235Z,-3.602334,120.363824,271012
4,2018-12-18T11:20:30.365620Z,4774,0efc0594-dbbf-4f9a-b0b0-a488cfddf8a2,df1042ab-13e6-4072-b9d2-64a81974c51a,"[{'product_id': 55220, 'quantity': 1, 'item_pr...",Credit Card,Success,0,NaN,0,2018-12-23T17:24:07.361785Z,-3.602334,120.363824,198753


In [23]:
transaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 852584 entries, 0 to 852583
Data columns (total 14 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   created_at              852584 non-null  object 
 1   customer_id             852584 non-null  int64  
 2   booking_id              852584 non-null  object 
 3   session_id              852584 non-null  object 
 4   product_metadata        852584 non-null  object 
 5   payment_method          852584 non-null  object 
 6   payment_status          852584 non-null  object 
 7   promo_amount            852584 non-null  int64  
 8   promo_code              326536 non-null  object 
 9   shipment_fee            852584 non-null  int64  
 10  shipment_date_limit     852584 non-null  object 
 11  shipment_location_lat   852584 non-null  float64
 12  shipment_location_long  852584 non-null  float64
 13  total_amount            852584 non-null  int64  
dtypes: float64(2), int64

In [24]:
transaction = transaction.join(pd.json_normalize(transaction['product_metadata']))

In [25]:
def safe_json_parser(x):
    if pd.isna(x) or x in ["", "null", "None"]:  # Handle missing/invalid values
        return []
    try:
        return json.loads(x)  # Try standard JSON parsing
    except json.JSONDecodeError:
        try:
            return ast.literal_eval(x)  # Fallback for improperly formatted JSON
        except (ValueError, SyntaxError):
            return []  # Return empty list if all else fails

In [26]:
transaction["product_metadata"] = transaction["product_metadata"].astype(str).apply(safe_json_parser)

In [27]:
tran= transaction.explode('product_metadata')

In [28]:
tran = pd.concat([tran.drop(columns=["product_metadata"]),tran["product_metadata"].apply(pd.Series)], axis=1)

In [29]:
tran.head()

,created_at,customer_id,booking_id,session_id,payment_method,payment_status,promo_amount,promo_code,shipment_fee,shipment_date_limit,shipment_location_lat,shipment_location_long,total_amount,product_id,quantity,item_price
0,2018-07-29T15:22:01.458193Z,5868,186e2bee-0637-4710-8981-50c2d737bc42,3abaa6ce-e320-4e51-9469-d9f3fa328e86,Debit Card,Success,1415,WEEKENDSERU,10000,2018-08-03T05:07:24.812676Z,-8.227893,111.969107,199832,54728,1,191247
1,2018-07-30T12:40:22.365620Z,4774,caadb57b-e808-4f94-9e96-8a7d4c9898db,2ee5ead1-f13e-4759-92df-7ff48475e970,Credit Card,Success,0,NaN,10000,2018-08-03T01:29:03.415705Z,3.013470,107.802514,155526,16193,1,145526
2,2018-09-15T11:51:17.365620Z,4774,6000fffb-9c1a-4f4a-9296-bc8f6b622b50,93325fb6-eb00-4268-bb0e-6471795a0ad0,OVO,Success,0,NaN,10000,2018-09-18T08:41:49.422380Z,-2.579428,115.743885,550696,53686,4,135174
3,2018-11-01T11:23:48.365620Z,4774,f5e530a7-4350-4cd1-a3bc-525b5037bcab,bcad5a61-1b67-448d-8ff4-781d67bc56e4,Credit Card,Success,0,NaN,0,2018-11-05T17:42:27.954235Z,-3.602334,120.363824,271012,20228,1,271012
4,2018-12-18T11:20:30.365620Z,4774,0efc0594-dbbf-4f9a-b0b0-a488cfddf8a2,df1042ab-13e6-4072-b9d2-64a81974c51a,Credit Card,Success,0,NaN,0,2018-12-23T17:24:07.361785Z,-3.602334,120.363824,198753,55220,1,198753


In [30]:
tran['product_sales']=tran['quantity']*tran['item_price']

In [31]:
total_orders=tran.groupby('product_id')['quantity'].sum().reset_index(name='total_orders')         

In [32]:
successful_orders=tran[tran['payment_status']=='Success'].groupby('product_id')[['quantity','total_amount']].sum().reset_index()
successful_orders.rename(columns={'quantity': 'successful_orders', 'total_amount': 'total_sales'}, inplace=True)

In [33]:
avg_quantity = tran.groupby('product_id')['quantity'].mean().reset_index()

In [34]:
avg_item_price = tran.groupby('product_id')['item_price'].mean().reset_index()

In [35]:
total_promo_discount = tran.groupby('product_id')['promo_amount'].sum().reset_index()                                 

In [36]:
promo_usage = tran.groupby('product_id').apply(lambda x: (x['promo_amount'] > 0).mean()).reset_index(name='promo_usage_ratio')

/var/folders/4s/v2k7bm2d7vl1z46yc265hz3c0000gn/T/ipykernel_84198/1029309369.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  promo_usage = tran.groupby('product_id').apply(lambda x: (x['promo_amount'] > 0).mean()).reset_index(name='promo_usage_ratio')


In [37]:
avg_shipment_fee = tran.groupby('product_id')['shipment_fee'].mean().reset_index()

In [38]:
tran['created_at'] = pd.to_datetime(tran['created_at'])
tran['shipment_date_limit'] = pd.to_datetime(tran['shipment_date_limit'])
tran['shipment_days'] = (tran['shipment_date_limit'] - tran['created_at']).dt.days

In [39]:
avg_shipment_days = tran.groupby('product_id')['shipment_days'].mean().reset_index()

In [40]:
unique_customers = tran.groupby('product_id')['customer_id'].nunique().reset_index()

In [41]:
final_tran=total_orders.merge(successful_orders, on='product_id')\
                      .merge(avg_quantity, on='product_id')\
                      .merge(avg_item_price, on='product_id')\
                      .merge(total_promo_discount, on='product_id')\
                      .merge(promo_usage, on='product_id')\
                      .merge(avg_shipment_fee, on='product_id')\
                      .merge(avg_shipment_days, on='product_id')\
                      .merge(unique_customers, on='product_id')

In [42]:
final_tran.columns = ['product_id', 'total_orders', 'successful_orders', 'total_sales','avg_quantity', 'avg_item_price',
                    'total_promo_discount', 'promo_usage_ratio', 'avg_shipment_fee',
                    'avg_shipment_days', 'unique_customers']

In [43]:
final_tran                          

,product_id,total_orders,successful_orders,total_sales,avg_quantity,avg_item_price,total_promo_discount,promo_usage_ratio,avg_shipment_fee,avg_shipment_days,unique_customers
0,1163,39,38,22754679,1.392857,248495.392857,46110,0.285714,6607.142857,3.392857,28
1,1164,29,27,19505831,1.260870,245078.434783,46833,0.347826,9347.826087,2.434783,23
2,1165,31,30,33326089,1.033333,249508.366667,71282,0.433333,9333.333333,3.000000,30
3,1525,47,47,49457361,1.516129,269841.935484,38829,0.225806,9677.419355,3.000000,31
4,1526,31,30,22412092,1.240000,250362.680000,83114,0.560000,12800.000000,2.800000,25
...,...,...,...,...,...,...,...,...,...,...,...
44441,59995,26,26,18278184,1.444444,266591.444444,35818,0.444444,10277.777778,3.333333,18
44442,59996,29,27,30899003,1.000000,261103.586207,30529,0.241379,8965.517241,3.206897,29
44443,59998,34,32,30991209,1.214286,245314.142857,40916,0.285714,8928.571429,2.607143,28
44444,59999,31,31,24796665,1.192308,210652.346154,42924,0.307692,7307.692308,2.961538,26


In [44]:
product.head() 

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,Unnamed: 10,Unnamed: 11
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt,NaN,NaN
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans,NaN,NaN
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch,NaN,NaN
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants,NaN,NaN
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt,NaN,NaN


In [45]:
product.rename(columns={'id': 'product_id'}, inplace=True)

In [46]:
product_summary=final_tran.merge(product[['product_id','productDisplayName','masterCategory','articleType','baseColour','gender','season']],on="product_id",how="left")

In [47]:
product_summary.head()

,product_id,total_orders,successful_orders,total_sales,avg_quantity,avg_item_price,total_promo_discount,promo_usage_ratio,avg_shipment_fee,avg_shipment_days,unique_customers,productDisplayName,masterCategory,articleType,baseColour,gender,season
0,1163,39,38,22754679,1.392857,248495.392857,46110,0.285714,6607.142857,3.392857,28,Nike Sahara Team India Fanwear Round Neck Jersey,Apparel,Tshirts,Blue,Men,Summer
1,1164,29,27,19505831,1.260870,245078.434783,46833,0.347826,9347.826087,2.434783,23,Nike Men Blue T20 Indian Cricket Jersey,Apparel,Tshirts,Blue,Men,Winter
2,1165,31,30,33326089,1.033333,249508.366667,71282,0.433333,9333.333333,3.000000,30,Nike Mean Team India Cricket Jersey,Apparel,Tshirts,Blue,Men,Summer
3,1525,47,47,49457361,1.516129,269841.935484,38829,0.225806,9677.419355,3.000000,31,Puma Deck Navy Blue Backpack,Accessories,Backpacks,Navy Blue,Unisex,Fall
4,1526,31,30,22412092,1.240000,250362.680000,83114,0.560000,12800.000000,2.800000,25,Puma Big Cat Backpack Black,Accessories,Backpacks,Black,Unisex,Fall


In [48]:
# product_summary.to_csv('product_summary.csv',index=False)

# Customer Metrics

In [49]:
transaction.head()

,created_at,customer_id,booking_id,session_id,product_metadata,payment_method,payment_status,promo_amount,promo_code,shipment_fee,shipment_date_limit,shipment_location_lat,shipment_location_long,total_amount
0,2018-07-29T15:22:01.458193Z,5868,186e2bee-0637-4710-8981-50c2d737bc42,3abaa6ce-e320-4e51-9469-d9f3fa328e86,"[{'product_id': 54728, 'quantity': 1, 'item_pr...",Debit Card,Success,1415,WEEKENDSERU,10000,2018-08-03T05:07:24.812676Z,-8.227893,111.969107,199832
1,2018-07-30T12:40:22.365620Z,4774,caadb57b-e808-4f94-9e96-8a7d4c9898db,2ee5ead1-f13e-4759-92df-7ff48475e970,"[{'product_id': 16193, 'quantity': 1, 'item_pr...",Credit Card,Success,0,NaN,10000,2018-08-03T01:29:03.415705Z,3.013470,107.802514,155526
2,2018-09-15T11:51:17.365620Z,4774,6000fffb-9c1a-4f4a-9296-bc8f6b622b50,93325fb6-eb00-4268-bb0e-6471795a0ad0,"[{'product_id': 53686, 'quantity': 4, 'item_pr...",OVO,Success,0,NaN,10000,2018-09-18T08:41:49.422380Z,-2.579428,115.743885,550696
3,2018-11-01T11:23:48.365620Z,4774,f5e530a7-4350-4cd1-a3bc-525b5037bcab,bcad5a61-1b67-448d-8ff4-781d67bc56e4,"[{'product_id': 20228, 'quantity': 1, 'item_pr...",Credit Card,Success,0,NaN,0,2018-11-05T17:42:27.954235Z,-3.602334,120.363824,271012
4,2018-12-18T11:20:30.365620Z,4774,0efc0594-dbbf-4f9a-b0b0-a488cfddf8a2,df1042ab-13e6-4072-b9d2-64a81974c51a,"[{'product_id': 55220, 'quantity': 1, 'item_pr...",Credit Card,Success,0,NaN,0,2018-12-23T17:24:07.361785Z,-3.602334,120.363824,198753


In [50]:
s_tran= transaction[transaction['payment_status'] == 'Success']

In [51]:
purchase_frequency= s_tran.groupby('customer_id').size()

In [52]:
customer_type=purchase_frequency.apply(lambda x: 'new' if x == 1 else 'repeated')

In [53]:
total_spent = s_tran.groupby('customer_id')['total_amount'].sum()

In [54]:
average_order_value = total_spent / purchase_frequency

In [55]:
discount_codes = s_tran.groupby('customer_id')['promo_code'].apply(lambda x: ', '.join(x.dropna().unique()))

In [56]:
total_promo_amount = s_tran.groupby('customer_id')['promo_amount'].sum()

In [57]:
customer_metrics = pd.DataFrame({
    'customer_id': purchase_frequency.index,
    'purchase_frequency': purchase_frequency,
    'customer_type': customer_type,
    'total_spent': total_spent,
    'average_order_value': average_order_value,
    'discount_codes': discount_codes,
    'total_promo_amount': total_promo_amount
})

In [58]:
customer_metrics.reset_index(drop=True, inplace=True)

In [59]:
customer_metrics

,customer_id,purchase_frequency,customer_type,total_spent,average_order_value,discount_codes,total_promo_amount
0,3,49,repeated,20891374,426354.571429,"WEEKENDMANTAP, WEEKENDSERU, LIBURDONG",125060
1,8,7,repeated,3898561,556937.285714,WEEKENDSERU,7699
2,9,6,repeated,2638665,439777.500000,,0
3,11,1,new,197533,197533.000000,XX2022,9492
4,15,5,repeated,2134870,426974.000000,"BUYMORE, WEEKENDMANTAP, XX2022",16362
...,...,...,...,...,...,...,...
50237,99992,12,repeated,4352229,362685.750000,AZ2022,12442
50238,99995,59,repeated,31437103,532832.254237,"AZ2022, WEEKENDMANTAP, XX2022, BUYMORE, STARTU...",63674
50239,99997,1,new,226461,226461.000000,,0
50240,99998,8,repeated,6692540,836567.500000,"AZ2022, LIBURDONG, WEEKENDMANTAP, BUYMORE",30325


In [60]:
customer.head()

,customer_id,first_name,last_name,username,email,gender,birthdate,device_type,device_id,device_version,home_location_lat,home_location_long,home_location,home_country,first_join_date
0,2870,Lala,Maryati,671a0865-ac4e-4dc4-9c4f-c286a1176f7e,671a0865_ac4e_4dc4_9c4f_c286a1176f7e@startupca...,F,1996-06-14,iOS,c9c0de76-0a6c-4ac2-843f-65264ab9fe63,iPhone; CPU iPhone OS 14_2_1 like Mac OS X,-1.043345,101.360523,Sumatera Barat,Indonesia,2019-07-21
1,8193,Maimunah,Laksmiwati,83be2ba7-8133-48a4-bbcb-b46a2762473f,83be2ba7_8133_48a4_bbcb_b46a2762473f@zakyfound...,F,1993-08-16,Android,fb331c3d-f42e-40fe-afe2-b4b73a8a6e25,Android 2.2.1,-6.212489,106.818850,Jakarta Raya,Indonesia,2017-07-16
2,7279,Bakiman,Simanjuntak,3250e5a3-1d23-4675-a647-3281879d42be,3250e5a3_1d23_4675_a647_3281879d42be@startupca...,M,1989-01-23,iOS,d13dde0a-6ae1-43c3-83a7-11bbb922730b,iPad; CPU iPad OS 4_2_1 like Mac OS X,-8.631607,116.428436,Nusa Tenggara Barat,Indonesia,2020-08-23
3,88813,Cahyadi,Maheswara,df797edf-b465-4a80-973b-9fbb612260c2,df797edf_b465_4a80_973b_9fbb612260c2@zakyfound...,M,1991-01-05,iOS,f4c18515-c5be-419f-8142-f037be47c9cd,iPad; CPU iPad OS 14_2 like Mac OS X,1.299332,115.774934,Kalimantan Timur,Indonesia,2021-10-03
4,82542,Irnanto,Wijaya,36ab08e1-03de-42a8-9e3b-59528c798824,36ab08e1_03de_42a8_9e3b_59528c798824@startupca...,M,2000-07-15,iOS,e46e4c36-4630-4736-8fcf-663db29ca3b0,iPhone; CPU iPhone OS 10_3_3 like Mac OS X,-2.980807,114.924675,Kalimantan Selatan,Indonesia,2021-04-11


In [61]:
customer_metrics=customer_metrics.merge(customer[['customer_id','first_name','last_name','email','gender','birthdate','device_type','device_id',
                                                  'device_version','home_location','home_country','first_join_date']],on='customer_id',how='left')

In [62]:
customer_metrics.head()

,customer_id,purchase_frequency,customer_type,total_spent,average_order_value,discount_codes,total_promo_amount,first_name,last_name,email,gender,birthdate,device_type,device_id,device_version,home_location,home_country,first_join_date
0,3,49,repeated,20891374,426354.571429,"WEEKENDMANTAP, WEEKENDSERU, LIBURDONG",125060,Ganep,Mustofa,76412a05_64ab_4e8c_b275_a7181b5c0b9d@startupca...,M,1986-07-12,Android,dd0c9125-c645-4b82-8d4b-a1cd1e2dba20,Android 3.2.3,Jakarta Raya,Indonesia,2018-08-25
1,8,7,repeated,3898561,556937.285714,WEEKENDSERU,7699,Nilam,Fujiati,848e59ba_e4f7_480a_bd36_e31fd4a03f83@startupca...,F,2000-07-19,Android,f8a01a56-a55f-4a33-9ad9-2fa69137f7ff,Android 2.2,Sulawesi Utara,Indonesia,2021-02-13
2,9,6,repeated,2638665,439777.500000,,0,Hesti,Purwanti,8abe001b_5d2e_4af1_8c12_0b44309d38ca@startupca...,F,2006-04-05,Android,ece472e7-ab36-4d85-aca0-2df8491189fe,Android 5.1.1,Jakarta Raya,Indonesia,2021-06-26
3,11,1,new,197533,197533.000000,XX2022,9492,Luwar,Permadi,5771c240_e53e_448a_8938_b1fc7a2c120f@zakyfound...,M,1993-07-27,Android,8c8aa601-df75-473a-aaeb-c8d027d543d9,Android 4.3,Kalimantan Timur,Indonesia,2022-03-06
4,15,5,repeated,2134870,426974.000000,"BUYMORE, WEEKENDMANTAP, XX2022",16362,Tari,Hassanah,fa713b1b_3f29_41a5_abba_bba5df3c9290@startupca...,F,1983-06-28,Android,aea56e94-eea4-462f-b717-bdf8722047b7,Android 4.3,Jawa Barat,Indonesia,2019-09-28


In [63]:
# customer_metrics.to_csv("customer_metrics.csv",index=False)

# Promotion Data

In [64]:
transaction.head()

,created_at,customer_id,booking_id,session_id,product_metadata,payment_method,payment_status,promo_amount,promo_code,shipment_fee,shipment_date_limit,shipment_location_lat,shipment_location_long,total_amount
0,2018-07-29T15:22:01.458193Z,5868,186e2bee-0637-4710-8981-50c2d737bc42,3abaa6ce-e320-4e51-9469-d9f3fa328e86,"[{'product_id': 54728, 'quantity': 1, 'item_pr...",Debit Card,Success,1415,WEEKENDSERU,10000,2018-08-03T05:07:24.812676Z,-8.227893,111.969107,199832
1,2018-07-30T12:40:22.365620Z,4774,caadb57b-e808-4f94-9e96-8a7d4c9898db,2ee5ead1-f13e-4759-92df-7ff48475e970,"[{'product_id': 16193, 'quantity': 1, 'item_pr...",Credit Card,Success,0,NaN,10000,2018-08-03T01:29:03.415705Z,3.013470,107.802514,155526
2,2018-09-15T11:51:17.365620Z,4774,6000fffb-9c1a-4f4a-9296-bc8f6b622b50,93325fb6-eb00-4268-bb0e-6471795a0ad0,"[{'product_id': 53686, 'quantity': 4, 'item_pr...",OVO,Success,0,NaN,10000,2018-09-18T08:41:49.422380Z,-2.579428,115.743885,550696
3,2018-11-01T11:23:48.365620Z,4774,f5e530a7-4350-4cd1-a3bc-525b5037bcab,bcad5a61-1b67-448d-8ff4-781d67bc56e4,"[{'product_id': 20228, 'quantity': 1, 'item_pr...",Credit Card,Success,0,NaN,0,2018-11-05T17:42:27.954235Z,-3.602334,120.363824,271012
4,2018-12-18T11:20:30.365620Z,4774,0efc0594-dbbf-4f9a-b0b0-a488cfddf8a2,df1042ab-13e6-4072-b9d2-64a81974c51a,"[{'product_id': 55220, 'quantity': 1, 'item_pr...",Credit Card,Success,0,NaN,0,2018-12-23T17:24:07.361785Z,-3.602334,120.363824,198753


In [65]:
transaction['created_at'] = pd.to_datetime(transaction['created_at'])

In [66]:
s_tran= transaction[transaction['payment_status'] == 'Success']
# s_tran= transaction

In [67]:
promo_summary = s_tran.groupby('promo_code').agg(
    frequency=('promo_code', 'count'),
    start_date=('created_at', 'min'),
    end_date=('created_at', 'max'),
    total_sales=('total_amount', 'sum'),
    avg_promo_amount=('promo_amount', 'mean')
).reset_index()


In [68]:
promo_summary['duration'] = (promo_summary['end_date'] - promo_summary['start_date']).dt.days

In [69]:
promo_summary = promo_summary[['promo_code', 'frequency', 'start_date', 'end_date', 'duration', 'total_sales', 'avg_promo_amount']]

In [70]:
promo_summary

,promo_code,frequency,start_date,end_date,duration,total_sales,avg_promo_amount
0,AZ2022,85396,2016-07-04 00:58:13.525069+00:00,2022-07-29 23:51:29.261426+00:00,2216,46833229484,4987.240000
1,BUYMORE,64029,2016-07-04 03:19:37.878958+00:00,2022-07-29 23:54:29.507639+00:00,2216,34930351637,4986.606647
2,LIBURDONG,20019,2016-07-09 02:12:25.213951+00:00,2022-07-31 23:57:43.676395+00:00,2213,11083440791,4996.390479
3,SC2022,10801,2016-07-04 11:51:57.313193+00:00,2022-07-29 19:22:10.590991+00:00,2216,5932555380,9958.651884
4,STARTUP,10630,2016-07-12 23:48:02.850252+00:00,2022-07-29 23:14:01.951340+00:00,2207,5907419400,9994.781938
5,WEEKENDMANTAP,19587,2016-07-03 05:54:12.839715+00:00,2022-07-31 23:39:12.239014+00:00,2219,10640780451,4994.055190
6,WEEKENDSERU,59293,2016-07-02 14:48:55.495757+00:00,2022-07-31 23:58:53.821469+00:00,2220,32498800608,4998.490243
7,XX2022,42844,2016-07-01 02:53:26.720195+00:00,2022-07-29 23:23:20.906511+00:00,2219,23406312333,4985.907362


In [71]:
# promo_summary.to_csv("promo_summary.csv",index=False)